In [121]:
import jinja2
import os
import cv2
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, Tag

In [122]:
def describe_images(folder):
    desc=[]
    for i in os.listdir(folder):
        image =folder+'/'+i
        #print(image)
        img =  cv2.imread(image)
        height, width, depth = img.shape
        desc.append({'file':image,'w':width,'h':height,'aspect_ratio':height/width})
        
    desc=pd.DataFrame(desc)
    
    return desc

In [124]:
thumbs=describe_images('images/tmp_density/thumbs')
thumbs.columns=['tfile','tw','th','taspect_ratio']

In [125]:
fulls=describe_images('images/tmp_density/fulls')

In [126]:
html = open("density.html").read()
soup = BeautifulSoup(html)
filtered = soup.find_all('article')

tags=[]
for f in filtered[1:]:
    tags.append({
        'class':' '.join(f.get("class")),
        'full': f.find("a").get("href"),
        'thumb':f.find("img").get("src")
        
    })

In [127]:
tags=pd.DataFrame(tags)

In [128]:
tags=tags.merge(fulls,left_on='full',right_on='file',how='left').merge(thumbs,left_on='thumb',right_on='tfile',how='left')

In [129]:
tags=tags.head(4)

In [130]:
def match(source,model):
    df=model.copy()
    src=source.copy()
    idxs=[]
    for index, row in df.iterrows():
        aspect_ratio=row['aspect_ratio']
        src2=src.iloc[(src['aspect_ratio']-aspect_ratio).abs().argsort()[:1]] #closest aspect ratio in source

        src=src.drop(src2.index[0],axis=0)
            #print(df.shape[0])
        idxs.append(src2.index[0])
        
    model['srcidx']=idxs
    src=source.copy()
    src=src.rename(columns={'file':'sfile','w':'sw','h':'sh','aspect_ratio':'saspect_ratio'})
    src['srcidx']=src.index
    model=model.merge(src,on='srcidx',how='left')
        
    return model
        

In [132]:
source=describe_images('images/Francisco Humeres - Clustering Experiments_files2')

In [133]:
tags=match(source,tags)

In [134]:
def resize(img,ar,w,h):
    height, width, depth = img.shape
    aspect_ratio = ar #actual aspect ratio
    print('original size ---> height: '+str(height)+' width: '+str(width))
    if height>width:
        dimension = (int(h/aspect_ratio),h)
    else:
        dimension = (w,int(w*aspect_ratio))
    
    print('new size ---> height: '+str(dimension[0])+' width: '+str(dimension[1]))
    return cv2.resize(img, dimension)
    #return dimension

In [135]:
def caption(img,caption,x,y):
    height, width, depth = img.shape
    
    BLACK = (255,255,255)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_size = 0.5
    font_color = BLACK
    font_thickness = 1
    text = caption
    x,y = int(width*x),int(height*y)
    img = cv2.putText(img, text, (x,y), font, font_size, font_color, font_thickness, cv2.LINE_AA)
    
    return img

In [136]:
def thumbnail(img,ar,w,h):
    height, width, depth = img.shape
    print('original image dim---> height:'+str(height)+' width: '+str(width))
    aspect_ratio = ar #actual aspect ratio
    if width>height:        
        if int(w*aspect_ratio)>h:
            dimension=(w,int(w*aspect_ratio))
        else:
            dimension=(int(h/aspect_ratio),h)
    else:
        if int(w*aspect_ratio)>h:
            dimension=(w,int(w*aspect_ratio))
        else:
            dimension=(int(h/aspect_ratio),h)
    print(dimension)
    img=cv2.resize(img, dimension)
    height, width, depth = img.shape
    print('resized image dim---> height:'+str(height)+' width: '+str(width))    
    

    img=img[
            int(height/2)-int(h/2):int(height/2)+int(h/2),
    int(width/2)-int(w/2):int(width/2)+int(w/2)]
    height, width, depth = img.shape
    print('final thumbnail dim---> height:'+str(height)+' width: '+str(width))
    return img

In [137]:
def batch_resize(df,target='',thumb_target='',cap=''):
    tmptfile=[]
    tmptw=[]
    tmpth=[]
    tmptaspect_ratio=[]
    
    tmpffile=[]
    tmpfw=[]
    tmpfh=[]
    tmpfaspect_ratio=[]
    
    for index, row in df.iterrows():
        i=row['sfile'].rsplit('/',1)[1]
        image =row['sfile']
        print('processing image:'+image)
        img =  cv2.imread(image)

        print('resizing image')
        aspect_ratio=row['saspect_ratio']
        w=row['w']
        h=row['h']
        img=resize(img,aspect_ratio,w,h)
        h, w, d = img.shape
        tmpffile.append(target+'/'+i)
        tmpfw.append(w)
        tmpfh.append(h)
        tmpfaspect_ratio.append(h/w)
        
        w=row['tw']
        h=row['th']
        print('generating thumbnail')
        thumb=thumbnail(img,aspect_ratio,w,h)
        h, w, d = thumb.shape
        tmptfile.append(thumb_target+'/'+i)
        tmptw.append(w)
        tmpth.append(h)
        tmptaspect_ratio.append(h/w)

        print('adding caption')
        img=caption(img,cap,0.07,0.95)
        print('saving file '+target+'/'+i)
        cv2.imwrite(target+'/'+i,img)
        print('saving file '+thumb_target+'/'+i)
        cv2.imwrite(thumb_target+'/'+i,thumb)
        
    df['tmptfile']=tmptfile
    df['tmptw']=tmptw
    df['tmpth']=tmpth
    df['tmptaspect_ratio']=tmptaspect_ratio
    
    df['tmpffile']=tmpffile
    df['tmpfw']=tmpfw
    df['tmpfh']=tmpfh
    df['tmpfaspect_ratio']=tmpfaspect_ratio
    
    df['tmp_thumb_filename']=df.tmptfile.str.split('/',-1,expand=True)[3]
    df['order']=df.tmp_thumb_filename.str.split('-',-1,expand=True)[0]
    df['caption']=df.tmp_thumb_filename.str.replace('_',' ').str.rsplit('.',1,expand=True)[0]
    
    return df

In [138]:
tags=batch_resize(tags,thumb_target='images/tmp_cluster/thumbs',target='images/tmp_cluster/fulls',cap='(c) 2013-14 Francisco J. Humeres M.')

processing image:images/Francisco Humeres - Clustering Experiments_files2/2-minneapolis.png
resizing image
original size ---> height: 914 width: 1072
new size ---> height: 525 width: 447
generating thumbnail
original image dim---> height:447 width: 525
(476, 405)
resized image dim---> height:405 width: 476
final thumbnail dim---> height:212 width: 476
adding caption
saving file images/tmp_cluster/fulls/2-minneapolis.png
saving file images/tmp_cluster/thumbs/2-minneapolis.png
processing image:images/Francisco Humeres - Clustering Experiments_files2/3-st_paul.png
resizing image
original size ---> height: 912 width: 1120
new size ---> height: 525 width: 427
generating thumbnail
original image dim---> height:427 width: 525
(476, 387)
resized image dim---> height:387 width: 476
final thumbnail dim---> height:212 width: 476
adding caption
saving file images/tmp_cluster/fulls/3-st_paul.png
saving file images/tmp_cluster/thumbs/3-st_paul.png
processing image:images/Francisco Humeres - Clusteri

In [139]:
def tags_order(tags):
    tags['order']=tags['order'].astype(int)
    tags=tags.sort_values(by='order')
    tags['switch']=np.floor(tags['order']/8).diff().diff().fillna(0)
    tags.loc[tags['order']==0,'switch']=0
    if tags.shape[0]>8:
        tags.loc[tags['order'].isin([7,8,9,10,11]),'switch']=[1,-1,0,0,0]
    tags['index']=tags['order']
    tags.index=tags['index']

    introw=274 #row de frame de intro
    tags['wcontrol']=tags['tmptw']
    #print(tags)
    
    first_row=tags.loc[0,'wcontrol']+introw
    
    tags.loc[0,'wcontrol']=first_row
    tags['wcontrol']=np.floor(tags['wcontrol'].cumsum()/tags.loc[0:7,'wcontrol'].sum()).diff().diff().fillna(0)
    #tags.loc[0,'wcontrol']=0
    #tags['wcontrol']=tags['switch']-tags['wcontrol']
    tags['switch']=tags['wcontrol']
    
        
    return tags

In [140]:
tags=tags_order(tags)

In [141]:
tags=tags[['switch','index','class','caption','tmpffile','tmptfile','order']].sort_values(by='order').rename(columns={'tmptfile':'thumb','tmpffile':'full'}).to_dict(orient='records')

In [142]:
tags

[{'switch': 0.0,
  'index': 0,
  'class': 'item thumb span-3',
  'caption': '0-buenos aires (yellow)-la plata (white)',
  'full': 'images/tmp_cluster/fulls/0-buenos_aires_(yellow)-la_plata_(white).png',
  'thumb': 'images/tmp_cluster/thumbs/0-buenos_aires_(yellow)-la_plata_(white).png',
  'order': 0},
 {'switch': 0.0,
  'index': 1,
  'class': 'item thumb span-2',
  'caption': '1-osaka (yellow)-kobe (red)',
  'full': 'images/tmp_cluster/fulls/1-osaka_(yellow)-kobe_(red).png',
  'thumb': 'images/tmp_cluster/thumbs/1-osaka_(yellow)-kobe_(red).png',
  'order': 1},
 {'switch': 0.0,
  'index': 2,
  'class': 'item thumb span-3',
  'caption': '2-minneapolis',
  'full': 'images/tmp_cluster/fulls/2-minneapolis.png',
  'thumb': 'images/tmp_cluster/thumbs/2-minneapolis.png',
  'order': 2},
 {'switch': 1.0,
  'index': 3,
  'class': 'item thumb span-2',
  'caption': '3-st paul',
  'full': 'images/tmp_cluster/fulls/3-st_paul.png',
  'thumb': 'images/tmp_cluster/thumbs/3-st_paul.png',
  'order': 3}]

In [143]:
from jinja2 import Template
with open('template.html') as file_:
    template = Template(file_.read())


In [144]:
rendered=template.render(tags=tags)

In [145]:
with open('cluster2.html', 'w') as f:
    f.write(rendered)